In [1]:
import os
import re
import cv2
import gdown
import numpy as np
import nibabel
import torch
import torch.utils.data
import pytorch_lightning as pl
from tqdm import tqdm
from typing import Any, Union, List, Optional
from os.path import isdir, join
from torch.utils.data.dataset import T_co
from torch.utils.data import Dataset, DataLoader, random_split
import sys
from k_space_reconstruction.utils.kspace import RandomMaskFunc, MaskFunc, spatial2kspace, kspace2spatial, apply_mask
from k_space_reconstruction.utils.io import get_dir_md5hash, get_file_md5hash
import pylab as plt
import configparser
from configobj import ConfigObj

In [2]:
import h5py
import numpy as np

def save_data_to_hdf5(file_path, patient_number, img, gt, cls):
    # Open the HDF5 file in write mode
    with h5py.File(file_path, 'a') as hdf5_file:
        # Create a group for each patient based on their number
        patient_group = hdf5_file.create_group(str(patient_number))

        # Store MRI image and segmentation image as datasets in the patient group
        patient_group.create_dataset('img', data=img)
        patient_group.create_dataset('gt', data=gt)

        # Store patient class as an attribute
        patient_group.attrs['class'] = cls


def load_scan_slice(f: str):
    scan = nibabel.load(f)
    images = []
    # print(scan.shape)
    for i in range(scan.shape[2]):
        # for j in range(scan.shape[3]):
        images.append((f, i, 0))
    return images

def read_config_file(file_path):
    config_data = {}

    with open(file_path, 'r') as file:
        for line in file:
            key, value = line.strip().split(': ')
            config_data[key] = value

    return config_data



dir_path = 'D:/OneDrive - University of Moratuwa/Volume D/7 sem 2023/MI/Implemetation/database/training'
hdf5_path = 'D:/OneDrive - University of Moratuwa/Volume D/7 sem 2023/MI/Implemetation/train.h5'
patients = [d for d in os.listdir(dir_path) if not d.startswith('.') and isdir(join(dir_path, d))]

images = []
ind = 0
for patient in sorted(patients):

    configFilePath = join(dir_path, patient,'Info.cfg')
    config_data = read_config_file(configFilePath)

    # Accessing values:
    print("ED:",config_data['ED'])
    print("ES:", config_data['ES'])
    print("Group:", config_data['Group'])


    firstScanIm = nibabel.load(join(dir_path, patient,  patient+'_frame'+config_data['ED'].zfill(2)+'.nii.gz')).dataobj[:, :, :]
    firstScanGT = nibabel.load(join(dir_path, patient,  patient+'_frame'+config_data['ED'].zfill(2)+'_gt.nii.gz')).dataobj[:, :, :]
    lastScanIm = nibabel.load(join(dir_path, patient,  patient+'_frame'+config_data['ES'].zfill(2)+'.nii.gz')).dataobj[:, :, :]
    lastScanGT = nibabel.load(join(dir_path, patient,  patient+'_frame'+config_data['ES'].zfill(2)+'_gt.nii.gz')).dataobj[:, :, :]

    print(firstScanIm.shape)  
    print(firstScanGT.shape)  
    print(lastScanIm.shape)  
    print(lastScanGT.shape) 
    
    for s in range(firstScanIm.shape[2]):
        
        img = cv2.resize(firstScanIm[:,:,s], (256, 256))
        gt = cv2.resize(firstScanGT[:,:,s], (256, 256))

        # fig, ax = plt.subplots(1,2, figsize=(5, 4 * 2), subplot_kw=dict(frameon=False, xticks=[], yticks=[]), gridspec_kw=dict(wspace=0.0, hspace=0.0))
        # ax[0].imshow(img)
        # ax[1].imshow(gt)

        save_data_to_hdf5(hdf5_path, str(ind), img, gt,  config_data['Group'])

        ind+=1

    for slice in range(lastScanIm.shape[2]):
        
        img = cv2.resize(lastScanIm[:,:,slice], (256, 256))
        gt = cv2.resize(lastScanGT[:,:,slice], (256, 256))
        
        # fig, ax = plt.subplots(1,2, figsize=(5, 4 * 2), subplot_kw=dict(frameon=False, xticks=[], yticks=[]), gridspec_kw=dict(wspace=0.0, hspace=0.0))
        # ax[0].imshow(img)
        # ax[1].imshow(gt)

        save_data_to_hdf5(hdf5_path, str(ind), img, gt,  config_data['Group'])
        
        ind+=1

    if ind>100:
        break

    
print(ind)

ED: 1
ES: 12
Group: DCM
(216, 256, 10)
(216, 256, 10)
(216, 256, 10)
(216, 256, 10)
ED: 1
ES: 12
Group: DCM
(232, 256, 10)
(232, 256, 10)
(232, 256, 10)
(232, 256, 10)
ED: 1
ES: 15
Group: DCM
(256, 256, 10)
(256, 256, 10)
(256, 256, 10)
(256, 256, 10)
ED: 1
ES: 15
Group: DCM
(232, 256, 10)
(232, 256, 10)
(232, 256, 10)
(232, 256, 10)
ED: 1
ES: 13
Group: DCM
(256, 216, 10)
(256, 216, 10)
(256, 216, 10)
(256, 216, 10)
ED: 1
ES: 16
Group: DCM
(232, 256, 11)
(232, 256, 11)
(232, 256, 11)
(232, 256, 11)
122


In [4]:
import h5py
import numpy as np



dir_path = 'D:/OneDrive - University of Moratuwa/Volume D/7 sem 2023/MI/Implemetation/database/testing'
hdf5_path = 'D:/OneDrive - University of Moratuwa/Volume D/7 sem 2023/MI/Implemetation/val.h5'
patients = [d for d in os.listdir(dir_path) if not d.startswith('.') and isdir(join(dir_path, d))]

images = []
ind = 0
for patient in sorted(patients):

    configFilePath = join(dir_path, patient,'Info.cfg')
    config_data = read_config_file(configFilePath)

    # Accessing values:
    print("ED:",config_data['ED'])
    print("ES:", config_data['ES'])
    print("Group:", config_data['Group'])


    firstScanIm = nibabel.load(join(dir_path, patient,  patient+'_frame'+config_data['ED'].zfill(2)+'.nii.gz')).dataobj[:, :, :]
    firstScanGT = nibabel.load(join(dir_path, patient,  patient+'_frame'+config_data['ED'].zfill(2)+'_gt.nii.gz')).dataobj[:, :, :]
    lastScanIm = nibabel.load(join(dir_path, patient,  patient+'_frame'+config_data['ES'].zfill(2)+'.nii.gz')).dataobj[:, :, :]
    lastScanGT = nibabel.load(join(dir_path, patient,  patient+'_frame'+config_data['ES'].zfill(2)+'_gt.nii.gz')).dataobj[:, :, :]

    print(firstScanIm.shape)  
    print(firstScanGT.shape)  
    print(lastScanIm.shape)  
    print(lastScanGT.shape) 
    
    for s in range(firstScanIm.shape[2]):
        
        img = cv2.resize(firstScanIm[:,:,s], (256, 256))
        gt = cv2.resize(firstScanGT[:,:,s], (256, 256))

        # fig, ax = plt.subplots(1,2, figsize=(5, 4 * 2), subplot_kw=dict(frameon=False, xticks=[], yticks=[]), gridspec_kw=dict(wspace=0.0, hspace=0.0))
        # ax[0].imshow(img)
        # ax[1].imshow(gt)

        save_data_to_hdf5(hdf5_path, str(ind), img, gt,  config_data['Group'])

        ind+=1

    for slice in range(lastScanIm.shape[2]):
        
        img = cv2.resize(lastScanIm[:,:,slice], (256, 256))
        gt = cv2.resize(lastScanGT[:,:,slice], (256, 256))
        
        # fig, ax = plt.subplots(1,2, figsize=(5, 4 * 2), subplot_kw=dict(frameon=False, xticks=[], yticks=[]), gridspec_kw=dict(wspace=0.0, hspace=0.0))
        # ax[0].imshow(img)
        # ax[1].imshow(gt)

        save_data_to_hdf5(hdf5_path, str(ind), img, gt,  config_data['Group'])
        
        ind+=1

    if ind>50:
        break

    
print(ind)

ED: 1
ES: 14
Group: DCM
(232, 256, 10)
(232, 256, 10)
(232, 256, 10)
(232, 256, 10)
ED: 1
ES: 13
Group: NOR
(216, 256, 8)
(216, 256, 8)
(216, 256, 8)
(216, 256, 8)
ED: 1
ES: 11
Group: MINF
(216, 256, 9)
(216, 256, 9)
(216, 256, 9)
(216, 256, 9)
54
